In [1]:
import pandas as pd
import numpy as np

if __name__ == "__main__":

  #read scan tsv into a dataframe
  raw_master_file_path = "/content/cbs_floor-s07-l01_scan-4_scan-analysis.tsv"
  raw_master_df = pd.read_csv(raw_master_file_path, sep="\t", skiprows=1) #skip the first empty row
  print("Raw Table: Raw Master")
  print(raw_master_df.head())

  # extract not built elements into a separate dataframe
  raw_not_built_master_columns = ["Not Built (IDs)"]
  raw_not_built_master_df = raw_master_df[raw_not_built_master_columns]
  print("Table 2: Raw Not Built Master")
  print(raw_not_built_master_df)

  # extract deviated elements into a separate dataframe
  raw_deviated_master_columns = ["Global Id", "Deviation X meters", "Deviation Y meters", "Deviation Z meters", "Deviation Meters", "Confidence"]
  raw_deviated_master_df = raw_master_df[raw_deviated_master_columns]
  print("Table 1: Raw Deviated Master")
  print(raw_deviated_master_df)

  # extract built elements into a dataframe
  raw_built_master_columns = ["In Place (IDs)"]
  raw_built_master_df = raw_master_df[raw_built_master_columns]
  print("Table 3: Raw Built Master")
  print(raw_built_master_df)

  # Read planned building element tsv into a dataframe, i.e. original_file_df
  original_file_path = "/content/cbs_floor-s07-l01_planned-building-elements.tsv"
  original_file_df = pd.read_csv(original_file_path, sep="\t")
  print("Planned Building Element TSV")
  print(original_file_df.head())

  # deviated to not built
  # by name, edit the name of the elements to be changed from deviated to built in betweeen the last set of inverted commas, e.g. "Ceiling"
  name_mask1 = original_file_df["Name"].str.contains("Floor")
  name_mask2 = original_file_df["Name"].str.contains("ceiling")
  # by uniformat, edit the uniformat of the elements to be changed from deviated to built in betweeen the last set of inverted commas, e.g. "B1010.10"
  uniformat_mask1 = original_file_df["Uniformat"].str.contains("C1010_10")
  name_and_uniformat_mask = name_mask1 | uniformat_mask1 | name_mask2
  deviated_to_not_built_df = original_file_df[name_and_uniformat_mask].reset_index(drop=True)
  print("deviated_to_not_built_df") 
  print(deviated_to_not_built_df)
  deviated_to_not_built_df_ids_only = raw_deviated_master_df["Global Id"].isin(deviated_to_not_built_df["Global ID"])
  deviated_to_not_built_df = raw_deviated_master_df[deviated_to_not_built_df_ids_only]
  print("deviated_to_not_built_df")
  print(deviated_to_not_built_df)

  #extract only the ids from deviated_to_not_built_df; remove other columns, i.e Name, Uniformat, etc
  deviated_to_not_built_ids = deviated_to_not_built_df["Global Id"].reset_index(drop=True)
  print("deviated_to_not_built_ids")
  print(deviated_to_not_built_ids)
  deviated_to_not_built_ids = deviated_to_not_built_ids.rename(level=0, index='Not Built (IDs)')
  deviated_to_not_built_ids = deviated_to_not_built_ids.to_frame() 
  print("deviated_to_not_built_ids")
  print(deviated_to_not_built_ids)
  #deviated_to_not_built_ids.to_csv("deviated_to_not_built_ids.tsv", sep="\t", index=False)

  #append deviated_to_not_built_ids to not_built_master_df
  not_built_master_df = raw_not_built_master_df.append(deviated_to_not_built_ids)
  not_built_master_df.replace('', np.nan, inplace=True)
  not_built_master_df.dropna(inplace=True)
  print("not_built_master_df")
  print(not_built_master_df)

  #filter out the deviated to built ids from raw_deviated_master_df
  deviated_master_bool = ~(raw_deviated_master_df["Global Id"].isin(deviated_to_not_built_ids["Not Built (IDs)"]))
  deviated_master_df = raw_deviated_master_df[deviated_master_bool]
  print(deviated_master_df)
  
  #concat all dataframes together
  final_df = pd.concat([
                        deviated_master_df.reset_index(inplace=False, drop=True),
                        pd.DataFrame(columns=[None, None, None]).reset_index(inplace=False, drop=True),
                        not_built_master_df.reset_index(inplace=False, drop=True),
                        pd.DataFrame(columns=[None]).reset_index(inplace=False, drop=True),
                        raw_built_master_df.reset_index(inplace=False, drop=True),
  ], axis=1)

  #append "Deviations from Design" into first cell of final_df
  multi_index_columns = [("Deviations from Design", (final_df.columns)[0])] + [("", column if column != None else "") for column in (final_df.columns)[1:]]
  final_df.columns = pd.MultiIndex.from_tuples(multi_index_columns)

  print("final_df")
  print(final_df)
  final_df.to_csv("final_df.tsv", sep="\t", index=False)

Raw Table: Raw Master
  Global Id  Deviation X meters  ...  Unnamed: 12  In Place (IDs)
0  Obj22781           -0.025000  ...          NaN         Obj6451
1  Obj24021           -0.090000  ...          NaN         Obj8351
2    Obj771            0.059157  ...          NaN        Obj21585
3  Obj15973           -0.025000  ...          NaN        Obj23539
4  Obj22619           -0.087500  ...          NaN        Obj21785

[5 rows x 14 columns]
Table 2: Raw Not Built Master
     Not Built (IDs)
0            Obj9999
1            Obj9997
2            Obj9995
3            Obj9975
4            Obj9973
...              ...
3246             NaN
3247             NaN
3248             NaN
3249             NaN
3250             NaN

[3251 rows x 1 columns]
Table 1: Raw Deviated Master
     Global Id  Deviation X meters  ...  Deviation Meters  Confidence
0     Obj22781           -0.025000  ...          0.136702         0.0
1     Obj24021           -0.090000  ...          0.101119         0.0
2       Obj77